### Question 1 (5 Marks)
In most DL applications, instead of training a model from scratch, you would use a model pre-trained on a similar/related task/dataset. From ```torchvision```, you can load **ANY ONE** [model](https://pytorch.org/vision/stable/models.html) (```GoogLeNet```, ```InceptionV3```, ```ResNet50```, ```VGG```, ```EfficientNetV2```, ```VisionTransformer``` etc.) pre-trained  on the ImageNet dataset. Given that ImageNet also contains many animal images, it stands to reason that using a model pre-trained on ImageNet maybe helpful for this task. 

You will load a pre-trained model and then fine-tune it using the naturalist data that you used in the previous question. Simply put, instead of randomly initialising the weigths of a network you will use the weights resulting from training the model on the ImageNet data (```torchvision``` directly provides these weights). Please answer the following questions:

- The dimensions of the images in your data may not be the same as that in the ImageNet data. How will you address this?
- ImageNet has $1000$ classes and hence the last layer of the pre-trained model would have $1000$ nodes. However, the naturalist dataset has only $10$ classes. How will you address this?

(Note: This question is only to check the implementation. The subsequent questions will talk about how exactly you will do the fine-tuning.)

In [4]:
#In the part A question, I have done preprocessing of the inaturalist dataset, by resizing the images to 256 X 256.. using the transform module of torchvision package.
#Based on few refereces, there is also an option to crop the centre of the image for usage.

from torchvision.datasets import ImageFolder
from torchvision import transforms
import torchvision.models as models
import torch.nn as nn
import os
import torchvision as tv
from torch.utils.data import DataLoader, random_split

model = models.googlenet(pretrained=True)

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

#To match the number of classes with the last layer nodes, I have replaced the final fully connected layer with a new layer that has only 10 neurons.

num_classes = 10
model.fc = nn.Linear(model.fc.in_features, num_classes)

### Question 2 (5 Marks)
You will notice that ```GoogLeNet```, ```InceptionV3```, ```ResNet50```, ```VGG```, ```EfficientNetV2```, ```VisionTransformer``` are very huge models as compared to the simple model that you implemented in Part A. Even fine-tuning on a small training data may be very expensive. What is a common trick used to keep the training tractable (you will have to read up a bit on this)? Try different variants of this trick and fine-tune the model using the iNaturalist dataset. For example, '_______'ing all layers except the last layer, '_______'ing upto $k$ layers and  '_______'ing the rest. Read up on pre-training and fine-tuning to understand what exactly these terms mean.

Write down the at least $3$ different strategies that you tried (simple bullet points would be fine).

Answer:
In order to reduce the cost of training, we can use the following three strategies:
- Feature Extraction: We can freeze all the layers except the last layer, so that we can preserve all the low level features and reduce the memory usage and for quick training
- Partial Finetuning: We can freeze the network parameters except for the last "k" layers, so that we can fine-tune much deeper. This is improve model performance on the current dataset. We can do this if we have enough compute to train efficiently. 
- Full Finetuning with discriminative learning rate: We can fine-tune the full network if we have enough compute. while doing this, we can have the learning rate of the early layers of the pretrained network lower, and have slightly larger learning rates for the last few layers. This way, the updation of the parameters in the early layers will happen slowly, kind of slowing down the "unlearning" of the trained parameters. But, I guess, this would be a bit expensive strategy. 

### Question 3 (10 Marks)
Now fine-tune the model using **ANY ONE** of the listed strategies that you discussed above. Based on these experiments write down some insightful inferences comparing training from scratch and fine-tuning a large pre-trained model.